In [1]:
from new_model.model_utils import resnet8, hybrid_LSTM_training, _classifier
from keras.layers import LSTM, Reshape
from keras.models import Model

Using TensorFlow backend.


# Global constant

In [2]:
IMG_SHAPE = (200, 200, 1)
LSTM_DIM_HIDDEN = 64
LEN_SPATIAL_HISTORY = 10
NUM_CLASS = 629

___

# 2.Model Definition

**------------------------**
## 2.1 Encoder

In [3]:
encoder = resnet8(IMG_SHAPE)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNormalization)     (None, 49, 49, 32)   128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
activation

**------------------------**
## 2.2 Decoder

### 2.2.1 Define LSTM

In [5]:
LSTM_cell = LSTM(LSTM_DIM_HIDDEN, return_state=True)

### 2.2.2 Define Classifier

In [6]:
classifier = _classifier(input_shape=(LSTM_DIM_HIDDEN, ), num_class=NUM_CLASS)

In [7]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 628)               40820     
_________________________________________________________________
activation_8 (Activation)    (None, 628)               0         
Total params: 40,820
Trainable params: 40,820
Non-trainable params: 0
_________________________________________________________________


In [8]:
reshapor = Reshape((1, -1))

## 2.2 Full Model

In [9]:
hybrid_model = hybrid_LSTM_training(IMG_SHAPE, encoder, LSTM_cell, LSTM_DIM_HIDDEN, classifier, reshapor, Ty=10)

In [10]:
hybrid_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 6272)         309088      input_3[0][0]                    
                                                                 input_4[0][0]                    
                                                                 input_5[0][0]                    
                                                                 input_6[0][0]                    
                                                                 input_7[0][0]                    
                                                                 input_8[0][0]                    
          